In [199]:
import json
import os
import random

# BASE_DIR = "/home/suyash/final_repo/WikiTableQuestions"
# BASE_DIR = "/home/suyash/final_repo/fetaqa_MM_cleaned"
# BASE_DIR = "/home/suyash/final_repo/WikiSQL"
BASE_DIR = "/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM"
random.seed(6969)

tables_dict = {}
questions_dict = {}

with open(os.path.join(BASE_DIR, "final_dataset_filtered", "tables.jsonl")) as f:
    for line in f.readlines():
        json_line = json.loads(line)
        tables_dict[json_line['table_id']] = json_line

with open(os.path.join(BASE_DIR, "final_dataset_filtered", "questions.jsonl")) as f:
    for line in f.readlines():
        json_line = json.loads(line)
        questions_dict[json_line['question_id']] = json_line

with open(os.path.join(BASE_DIR, "final_dataset_filtered", "implicit_questions_data.json")) as f:
    implicit_questions_data = json.load(f)

with open(os.path.join(BASE_DIR, "final_dataset_filtered", "explicit_questions_data.json")) as f:
    explicit_questions_data = json.load(f)

with open(os.path.join(BASE_DIR, "final_dataset_filtered", "explicit_answer_mention_data.json")) as f:
    explicit_answer_mention_data = json.load(f)

with open(os.path.join(BASE_DIR, "final_dataset_filtered", "image_id_to_original_string.json")) as f:
    image_id_to_original_string = json.load(f)

In [200]:
print(len(questions_dict))
print(len(implicit_questions_data))
print(len(explicit_questions_data))
print(len(explicit_answer_mention_data))

66148
19581
6319
5523


In [201]:
print(list(questions_dict.items())[:10])

[('00009b9649d0dd0a', {'question_id': '00009b9649d0dd0a', 'question': 'Who were the builders of the mosque in Herat with fire temples ?', 'answer-text': 'Ghurids', 'question_postag': 'WP VBD DT NNS IN DT NN IN NNP IN NN NNS .', 'answer-node': [['Kabul', [0, 2], '/wiki/Kabul', 'passage'], ['Kabul', [5, 2], '/wiki/Kabul', 'passage'], ['Great Mosque of Herat', [3, 0], '/wiki/Great_Mosque_of_Herat', 'passage'], ['Mazar-i-Sharif', [4, 2], '/wiki/Mazar-i-Sharif', 'passage']], 'table_context': 'List_of_mosques_in_Afghanistan_0'}), ('00013190d4370f73', {'question_id': '00013190d4370f73', 'question': 'After what season did the number 7 competitor retire ?', 'answer-text': '2006', 'question_postag': 'IN WP NN VBD DT NN CD NN NN .', 'answer-node': [['Pierfrancesco Chili', [4, 1], '/wiki/Pierfrancesco_Chili', 'passage']], 'table_context': '1999_Kyalami_Superbike_World_Championship_round_0'}), ('00023988273478d0', {'question_id': '00023988273478d0', 'question': 'What year was the 1971-72 ECAC Hocke

In [202]:
import re
def get_all_image_tags(text):
    pattern = r"\{IMG-\{.*?\}\}"
    matches = re.findall(pattern, text)
    return matches


def is_cell_image(text):
    return len(get_all_image_tags(text)) > 0


def generate_table_string(table_array):
    table_string = ""
    
    for row_idx, row in enumerate(table_array):
        for col_idx, cell in enumerate(row):
            cell = cell.replace("\t", " ").replace("\n", " ").replace("\\n", " ").replace("\\t", " ").replace("|", " ")
            table_string = table_string  + cell + " | "
        table_string = table_string + "\n"

    return table_string


def is_valid(table_array):
    if len(table_array) > 55:
        return False

    col_count = [0]*len(table_array[0])

    for row in table_array:
        for col_idx, cell in enumerate(row):
            if len(get_all_image_tags(cell)) > 0:
                col_count[col_idx] += 1
    
    col_count = [x/len(table_array) for x in col_count]

    if max(col_count) < 0.3:
        return False
    
    # if max(col_count) > 0.75:
    #     print("SED")

    return True

def filter_out_insufficient_images_tables(question_ids):
    filtered_question_ids = []
    for qid in question_ids:
        if BASE_DIR.split("/")[-1] != "fetaqa_MM_cleaned":
            question = questions_dict[(qid)]
        else:
            question = questions_dict[int(qid)]
        table_id = question['table_context']

        table_array = tables_dict[table_id]['table_array']

        if is_valid(table_array):
            filtered_question_ids.append(qid)

    return filtered_question_ids

print(len(implicit_questions_data))
filtered_implicit_question_ids = filter_out_insufficient_images_tables(implicit_questions_data)
print(len(filtered_implicit_question_ids))

print(len(explicit_answer_mention_data))
filtered_explicit_answer_mention_ids = filter_out_insufficient_images_tables(explicit_answer_mention_data)
print(len(filtered_explicit_answer_mention_ids))

print(len(explicit_questions_data))
filtered_explicit_question_ids = filter_out_insufficient_images_tables(explicit_questions_data.keys())
print(len(filtered_explicit_question_ids))

# FetaQA
    # 0.3:
        # 696  -> Implicit old
        # 578  -> Implicit new
        # 3446 -> Answer-mention old
        # 3181 -> Answer-mention new
        # 2663 -> Explicit old
        # 2543 -> Explicit new


# WikiTableQuestions
    # 0.3:
        # 3465 -> Implicit old
        # 2882 -> Implicit new
        # 3244 -> Answer-mention old
        # 2777 -> Answer-mention new
        # 4204 -> Explicit old
        # 3539 -> Explicit new

# WikiSQL
        # 339   -> Implicit old
        # 315   -> Implicit new
        # 6701  -> Answer-mention old
        # 6374  -> Answer-mention new
        # 13681 -> Explicit old
        # 12960 -> Explicit new

# HybridQA
        # 19581 -> Implicit old
        # 17640 -> Implicit new
        # 5523  -> Answer-mention old
        # 5130  -> Answer-mention new
        # 6319  -> Explicit old
        # 5848  -> Explicit new

19581


17640
5523
5130
6319
5848


In [203]:
# Check that all tables are unique
from collections import defaultdict
relevant_qids = filtered_implicit_question_ids + filtered_explicit_question_ids + filtered_explicit_answer_mention_ids
if BASE_DIR.split("/")[-1] != "fetaqa_MM_cleaned":
    relevant_qids = [(qid) for qid in relevant_qids]
else:
    relevant_qids = [int(qid) for qid in relevant_qids]


relevant_table_ids = [questions_dict[qid]['table_context'] for qid in relevant_qids]
og_table_set = defaultdict(list)

for table_id in relevant_table_ids:
    table_array = tables_dict[table_id]['original_table_array']
    table_array_tuple = tuple([tuple(row) for row in table_array])
    og_table_set[table_array_tuple].append(table_id)

table_id_to_finaltable = {}
finaltable_to_tables = defaultdict(list)
relevant_table_ids = []
for table_key, table_ids in og_table_set.items():
    table_ids = list(set(table_ids))
    for table_id in table_ids:
        table_id_to_finaltable[table_id] = table_ids[0]
    relevant_table_ids.append(table_ids[0])
    finaltable_to_tables[table_ids[0]] = table_ids

AAAAAAAAAAAAa
['List_of_cities_and_towns_in_India_10', 'List_of_populated_places_in_Adilabad_district_10']
(('Name of Town', 'State', 'Type', 'Population ( 2011 )'), ('Kadi', 'Gujarat', 'M', '73,228'), ('Kadiri', 'Andhra Pradesh', 'M', '89,429'), ('Kagaznagar', 'Telangana', 'M', '57,583'), ('Kailasahar', 'Tripura', 'N.P', '20,279'), ('Kaladhungi', 'Uttarakhand', 'N.P', '7,611'), ('Kallakurichi', 'Tamil Nadu', 'M', '1,57,628'), ('Kalimpong', 'West Bengal', 'M', '42,998'), ('Kalpi', 'Uttar Pradesh', 'NPP', '51,670'), ('Kamareddy', 'Telangana', 'M', '80,315'), ('Kanigiri', 'Andhra Pradesh', 'M', '37,420'), ('Kandukur', 'Andhra Pradesh', 'M', '57,246'), ('Kanhangad', 'Kerala', 'M', '73,342'), ('Kannur', 'Kerala', 'M', '56,823'), ('Kapadvanj', 'Gujarat', 'M', '49,308'), ('Karaikal', 'Puducherry', 'M', '86,838'), ('Karimganj', 'Assam', 'M.B', '56,854'), ('Karjat', 'Maharashtra', 'M.Cl', '29,663'), ('Karnaprayag', 'Uttarakhand', 'N.P', '8,297'), ('Karunagappally', 'Kerala', 'M', '72,634'), ('

In [204]:
def alternate_unique_elements(arrays):
    # Flatten the array to get all unique values upfront (for efficiency).
    try:
        all_uniques = set(elem for sub_array in arrays for elem in sub_array)
    except Exception as e:
        print(arrays)
        print(e)
        return []
    
    # Prepare a list to hold the final alternated unique elements
    alternated_uniques = []
    
    # We iterate as long as there are unique elements left to add.
    while all_uniques:
        # Track if we added at least one element in this iteration to know when to stop
        added_this_round = False
        
        for sub_array in arrays:
            # Attempt to find and add the first unique element in the current sub-array
            for elem in sub_array:
                if elem in all_uniques:
                    alternated_uniques.append(elem)
                    all_uniques.remove(elem)
                    added_this_round = True
                    break  # Move to the next sub-array after adding an element

        if not added_this_round:
            # If no elements were added in this iteration, break the loop to avoid an infinite loop
            break
    alternated_uniques.reverse()
    return alternated_uniques

# Example usage
arrays = [[1, 2, 3], [3, 4, 5, 8], [5, 6, 7]]
print(alternate_unique_elements(arrays))


[7, 8, 6, 4, 2, 5, 3, 1]


In [205]:
import math
from collections import defaultdict
import random
import copy



def create_tableid_to_evidence_map(qids):
    tableid_to_evidence = defaultdict(list)
    
    if BASE_DIR == "/home/suyash/final_repo/WikiTableQuestions":
        with open(os.path.join(BASE_DIR, "final_dataset_filtered", "qid_to_evidence_cells.json")) as f:
            qid_to_highlighted_cells = json.load(f)

    temp_evidence_cells = defaultdict(list)

    for qid in qids:
        table_id = questions_dict[qid]['table_context']
        final_table_id = table_id_to_finaltable[table_id]
        table_array = tables_dict[final_table_id]['table_array']

        if BASE_DIR.split("/")[-1] == 'WikiTableQuestions':
            try:
                evidence_cellls = qid_to_highlighted_cells[qid]
            except:
                print("Missing highlighted cells.")
                evidence_cellls = []
        elif BASE_DIR.split("/")[-1] == 'WikiSQL':
            evidence_cellls = questions_dict[qid]['weak_evidence_idx']
        elif BASE_DIR.split("/")[-1] == "Hybrid_QA_MM":
            evidence_cellls = questions_dict[qid]["answer-node"]
            evidence_cellls = [tuple([cell[1][0]+1, cell[1][1]]) for cell in evidence_cellls]
        else:
            evidence_cellls = questions_dict[qid]['highlighted_cell_ids']
        
        new_evidence_cells = []
        for cell in evidence_cellls:
            try:
                if is_cell_image(table_array[cell[0]][cell[1]]):
                    new_evidence_cells.append(cell)
            except:
                continue
        evidence_cellls = new_evidence_cells
        random.shuffle(evidence_cellls)
        evidence_cellls = [tuple(cell) for cell in evidence_cellls]
        
        temp_evidence_cells[final_table_id].append(tuple(evidence_cellls))
    
    final_evidence_cells = {}
    for table_id, temp_evidence in temp_evidence_cells.items():
        final_evidence_cells[table_id] = alternate_unique_elements(temp_evidence)
    
    return final_evidence_cells

def create_tableid_to_explicitcells_map():
    maybe_avoid_cells = defaultdict(list)
    definitely_avoid_cells = defaultdict(list)
    temp_maybe_avoid_cells = defaultdict(list)
    
    if BASE_DIR.split("/")[-1] == "WikiTableQuestions":
        with open(os.path.join(BASE_DIR, "final_dataset_filtered", "qid_to_evidence_cells.json")) as f:
            qid_to_highlighted_cells = json.load(f)
        
    expl_mention_ques = filtered_explicit_question_ids + filtered_explicit_answer_mention_ids
    expl_mention_data = {}
    expl_mention_data.update(explicit_questions_data)
    expl_mention_data.update(explicit_answer_mention_data)
    for qid in expl_mention_ques:
        explicit_question_data = expl_mention_data[str(qid)]
        explicit_cells = set()

        for _ in explicit_question_data:
            explicit_cells.add(tuple(_['cell_idx']))

        if BASE_DIR.split("/")[-1] != "fetaqa_MM_cleaned":
            questions_data = questions_dict[(qid)]
        else:
            questions_data = questions_dict[int(qid)]

        table_id = questions_data['table_context']

        if BASE_DIR.split("/")[-1] == "WikiTableQuestions":
            try:
                highlighted_cell = qid_to_highlighted_cells[qid]
            except:
                highlighted_cell = []
                print("Missing highlighted cells.")
        elif BASE_DIR.split("/")[-1] == "WikiSQL":
            highlighted_cell = questions_data['weak_evidence_idx']
        elif BASE_DIR.split("/")[-1] == "Hybrid_QA_MM":
            highlighted_cell = questions_dict[qid]["answer-node"]
            highlighted_cell = [tuple([cell[1][0]+1, cell[1][1]]) for cell in highlighted_cell]
        else:
            highlighted_cell = questions_data['highlighted_cell_ids'] #TODO: Fix this

        highlighted_cell_tuple = set(tuple([tuple(cell) for cell in highlighted_cell]))
        
        interestected_cells = highlighted_cell_tuple.intersection(explicit_cells)
        
        interestected_cells = set([cell for cell in interestected_cells if cell[0] != 0])
        
        if len(interestected_cells) > 0:
            pakka_avoid_cell = random.sample(interestected_cells, 1)[0]
        else:
            pakka_avoid_cell = random.sample(explicit_cells, 1)[0]



        interestected_cells = interestected_cells - set([pakka_avoid_cell])
        temp_maybe_avoid_cells[table_id].append(tuple(interestected_cells))

        definitely_avoid_cells[table_id].append(pakka_avoid_cell)


    for table_id, temp_evidence in temp_maybe_avoid_cells.items():
        maybe_avoid_cells[table_id] = alternate_unique_elements(temp_evidence)

    return maybe_avoid_cells, definitely_avoid_cells


def convert_to_text(text):
    img_ids = get_all_image_tags(text)
    for img_id in img_ids:
        text = text.replace(img_id, image_id_to_original_string[img_id])
    return text



tableid_to_maybeavoidcells, tableid_to_definitelyavoidcells = create_tableid_to_explicitcells_map()
tableid_to_evidence = create_tableid_to_evidence_map(relevant_qids)
filtered_tables = []


def fix_table(og_table_array, evidence_cells, maybe_avoid_cells, definitely_avoid_cells):
    table_array = copy.deepcopy(og_table_array)
    col_percents = [0]*len(table_array[0])
    
    for row_idx, row in enumerate(table_array):
        for col_idx, cell in enumerate(row):
            if row_idx == 0:
                table_array[row_idx][col_idx] = convert_to_text(table_array[row_idx][col_idx])
            elif len(get_all_image_tags(cell)) > 0:
                col_percents[col_idx] += 1
    
    col_nums = copy.deepcopy(col_percents)
    col_percents = [x/(len(table_array) - 1) for x in col_percents]

    
    if max(col_percents) <= 0.75:
        return table_array, False
    
    col_to_additional_textual_rows = defaultdict(float)
    col_to_maybe_avoid_rows = defaultdict(list)
    col_to_definitely_avoid_rows = defaultdict(list)
    col_to_evidence_cells = defaultdict(list)
    col_to_nonevidence_cells = defaultdict(list)

    for col_idx, percent in enumerate(col_percents):
        if percent > 0.75:
            more_images = math.ceil((percent - 0.75)*(len(table_array)-1))
            col_to_additional_textual_rows[col_idx] = more_images


    for cell in evidence_cells:
        if cell in maybe_avoid_cells or cell in definitely_avoid_cells:
            continue
        col_to_evidence_cells[cell[1]].append(cell[0])

    for cell in maybe_avoid_cells:
        if cell in definitely_avoid_cells:
            continue
        
        col_to_maybe_avoid_rows[cell[1]].append(cell[0])
    
    for cell in definitely_avoid_cells:
        col_to_definitely_avoid_rows[cell[1]].append(cell[0])
    
    for row_idx, row in enumerate(table_array):
        for col_idx, cell in enumerate(row):
            if row_idx == 0:
                continue

            if len(get_all_image_tags(cell)) == 0:
                continue

            if tuple([row_idx, col_idx]) in definitely_avoid_cells:
                continue
            elif tuple([row_idx, col_idx]) in maybe_avoid_cells:
                continue
            elif tuple([row_idx, col_idx]) in evidence_cells:
                continue
            else:
                col_to_nonevidence_cells[col_idx].append(row_idx)
    

    cells_to_convert = []
    
    for col_idx, num_additional_rows in col_to_additional_textual_rows.items():
        if len(col_to_nonevidence_cells[col_idx]) < num_additional_rows:
            col_to_nonevidence_cells[col_idx] += col_to_evidence_cells[col_idx][:min(num_additional_rows - len(col_to_nonevidence_cells[col_idx]), len(col_to_evidence_cells[col_idx]))]
            col_to_evidence_cells[col_idx] = list(set(col_to_evidence_cells[col_idx]) - set(col_to_nonevidence_cells[col_idx]))
        
        if len(col_to_nonevidence_cells[col_idx]) < num_additional_rows:
            print("CRIE", num_additional_rows - len(col_to_nonevidence_cells[col_idx]))
            col_to_nonevidence_cells[col_idx] += col_to_maybe_avoid_rows[col_idx][:min(num_additional_rows - len(col_to_nonevidence_cells[col_idx]), len(col_to_maybe_avoid_rows[col_idx]))]
            col_to_maybe_avoid_rows[col_idx] = list(set(col_to_maybe_avoid_rows[col_idx]) - set(col_to_nonevidence_cells[col_idx]))
        
        if len(col_to_nonevidence_cells[col_idx]) < num_additional_rows:
            print("WTFFF", num_additional_rows - len(col_to_nonevidence_cells[col_idx]))
            col_to_nonevidence_cells[col_idx] += col_to_definitely_avoid_rows[col_idx][:min(num_additional_rows - len(col_to_nonevidence_cells[col_idx]), len(col_to_definitely_avoid_rows[col_idx]))]
            col_to_definitely_avoid_rows[col_idx] = list(set(col_to_definitely_avoid_rows[col_idx]) - set(col_to_nonevidence_cells[col_idx]))

        rows_to_convert = random.sample(col_to_nonevidence_cells[col_idx], num_additional_rows)
        cells_to_convert.extend([(row_idx, col_idx) for row_idx in rows_to_convert])
    
    for row_idx, col_idx in cells_to_convert:
        if row_idx in col_to_definitely_avoid_rows[col_idx]:
            print("OMG!!")
        table_array[row_idx][col_idx] = convert_to_text(table_array[row_idx][col_idx])

    return table_array, True


/tmp/ipykernel_20394/681298337.py:109: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  pakka_avoid_cell = random.sample(explicit_cells, 1)[0]
/tmp/ipykernel_20394/681298337.py:107: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  pakka_avoid_cell = random.sample(interestected_cells, 1)[0]


In [206]:
from tqdm import tqdm
import copy

def get_relevant_images(table_array):
    table_string = generate_table_string(table_array)
    img_ids = get_all_image_tags(table_string)
    return img_ids

filtered_tables_dict = {}


for table_id in tqdm(relevant_table_ids):
    final_table_id = table_id_to_finaltable[table_id]
    if final_table_id in filtered_tables_dict:
        continue
    evidence_cells = tableid_to_evidence[final_table_id]
    maybe_avoid_cells = tableid_to_maybeavoidcells[final_table_id]
    definitely_avoid_cells = tableid_to_definitelyavoidcells[final_table_id]
    
    ogtable_array = tables_dict[final_table_id]['table_array']
    table_array = copy.deepcopy(ogtable_array)
    
    table_array, is_changed = fix_table(table_array, evidence_cells, maybe_avoid_cells, definitely_avoid_cells)
    
    filtered_tables_dict[final_table_id] = copy.deepcopy(tables_dict[final_table_id])
    filtered_tables_dict[final_table_id]['table_array'] = table_array


  8%|▊         | 644/8085 [00:00<00:02, 3222.88it/s]

CRIE 5
CRIE 1
CRIE 1


 26%|██▌       | 2067/8085 [00:00<00:01, 3534.18it/s]

CRIE 5
CRIE 2
CRIE 5


 44%|████▍     | 3542/8085 [00:01<00:01, 3655.22it/s]

...

CRIE 1
CRIE 3
CRIE 3
CRIE 1
CRIE 4
CRIE 1
CRIE 3
CRIE 5
CRIE 1


 94%|█████████▎| 7575/8085 [00:02<00:00, 3512.00it/s]

CRIE 2
CRIE 4
CRIE 2
CRIE 5
CRIE 1
CRIE 2
CRIE 1
CRIE 1
CRIE 4
CRIE 3
CRIE 1
CRIE 1
CRIE 3
CRIE 1
CRIE 1
CRIE 1
CRIE 2
CRIE 4
CRIE 1
CRIE 3
CRIE 4
CRIE 3
CRIE 1


100%|██████████| 8085/8085 [00:02<00:00, 3569.67it/s]

CRIE 2
CRIE 1
CRIE 3
CRIE 1
CRIE 4
CRIE 3


In [207]:
print(list(filtered_tables_dict.items())[:10])

[('List_of_car-free_places_0', {'table_id': 'List_of_car-free_places_0', 'table_idx': 7704, 'table_array': [['State', 'Location', 'Area', 'Character'], ['{IMG-{HTQ-7704-1-0-0}}', 'halibut cove', 'entire Town', "Originally a fishing village , Halibut Cove , located inside Kachemak Bay State Park , is now home to several art studios/galleries and businesses . One of the world 's only floating towns , it is a popular tourist destination . Only way to get around the cove is by boat . Outside access is by water taxis from Homer , 6 miles away"], ['{IMG-{HTQ-7704-2-0-0}}', 'arcosanti', 'entire town', 'Arcology being built by {IMG-{HTQ-7704-2-3-0}}'], ['arizona', 'phantom ranch', 'entire village', 'Tourist village on the Colorado River inside {IMG-{HTQ-7704-3-3-0}} ten miles ( plus one vertical mile ) from nearest road with access by foot , raft , or mule'], ['{IMG-{HTQ-7704-4-0-0}}', 'Supai , Arizona', 'entire town', 'In the havasupai Nation , the capital of Supai is eight miles from the nea

In [208]:
def create_final_questions(question_ids):
    questions_array = []
    for qid in question_ids:
        if BASE_DIR.split("/")[-1] != "fetaqa_MM_cleaned":
            question = questions_dict[(qid)]
        else:
            question = questions_dict[int(qid)]
        table_id = question['table_context']
        table_id = table_id_to_finaltable[table_id]
        question['table_context'] = table_id
        if table_id not in filtered_tables_dict:
            print("Table not found", table_id)
        questions_array.append(question)
    return questions_array

In [209]:

def create_final_implicit(qids, final_table_ids):
    filtered_qids = []
    if BASE_DIR.split("/")[-1] == "WikiTableQuestions":
        with open(os.path.join(BASE_DIR, "final_dataset_filtered", "qid_to_evidence_cells.json")) as f:
            qid_to_highlighted_cells = json.load(f)
    
    for qid in qids:
        if BASE_DIR.split("/")[-1] != "fetaqa_MM_cleaned":
            question = questions_dict[(qid)]
        else:
            question = questions_dict[int(qid)]

        table_id = question['table_context']
        table_id = table_id_to_finaltable[table_id]

        final_question = copy.deepcopy(question)
        final_question['table_context'] = table_id

        if table_id not in filtered_tables_dict:
            print("Table not found", table_id)
        
        if BASE_DIR.split("/")[-1] == "WikiTableQuestions":
            try:
                highlighted_cell = qid_to_highlighted_cells[qid]
            except:
                highlighted_cell = []
                print("Missing highlighted cells.")
        elif BASE_DIR.split("/")[-1] == "WikiSQL":
            highlighted_cell = questions_dict[qid]['weak_evidence_idx']
        elif BASE_DIR.split("/")[-1] == "Hybrid_QA_MM":
            highlighted_cell = questions_dict[qid]["answer-node"]
            highlighted_cell = [tuple([cell[1][0]+1, cell[1][1]]) for cell in highlighted_cell]
        else:
            highlighted_cell = question['highlighted_cell_ids']
        
        is_found = False
        for cell in highlighted_cell:
            try:
                if is_cell_image(filtered_tables_dict[table_id]['table_array'][cell[0]][cell[1]]):
                    is_found = True
                    break
            except:
                continue
        
        if is_found:
            filtered_qids.append(qid)
            final_table_ids.append(table_id)

    return filtered_qids

def create_final_explicit(qids, final_table_ids):
    with open(os.path.join(BASE_DIR, "final_dataset_filtered", "explicit_questions_data.json")) as f:
        og_explicit_questions_data = json.load(f)
    
    if BASE_DIR.split("/")[-1] == "WikiTableQuestions":
        with open(os.path.join(BASE_DIR, "final_dataset_filtered", "qid_to_evidence_cells.json")) as f:
            qid_to_highlighted_cells = json.load(f)
    expl_data = {}
    expl_data.update(explicit_questions_data)
    expl_data.update(explicit_answer_mention_data)
    filtered_qids = []
    for qid in qids:
        explicit_question_data = expl_data[str(qid)]
        explicit_cells = set()
        
        for _ in explicit_question_data:
            explicit_cells.add(tuple(_['cell_idx']))
        
        if BASE_DIR.split("/")[-1] != "fetaqa_MM_cleaned":
            questions_data = questions_dict[(qid)]
        else:
            questions_data = questions_dict[int(qid)]
        
        table_id = questions_data['table_context']
        final_table_id = table_id_to_finaltable[table_id]
        
        for cell in explicit_cells:
            try:
                if is_cell_image(filtered_tables_dict[final_table_id]['table_array'][cell[0]][cell[1]]):
                    filtered_qids.append(qid)
                    final_table_ids.append(final_table_id)
                    break
            except:
                continue
    
    return filtered_qids

In [210]:
final_table_ids = []

final_explicit_qids = create_final_explicit(filtered_explicit_question_ids, final_table_ids)
final_explicit_ansmention_qids = create_final_explicit(filtered_explicit_answer_mention_ids, final_table_ids)

left_explicit_qids = list(set(filtered_explicit_question_ids) - set(final_explicit_qids))
left_explicit_qids = left_explicit_qids + list(set(filtered_explicit_answer_mention_ids) - set(final_explicit_ansmention_qids))

tentative_implicit_qids = left_explicit_qids + filtered_implicit_question_ids

final_implicit_qids = create_final_implicit(tentative_implicit_qids, final_table_ids)
final_tables = {k: v for k, v in filtered_tables_dict.items() if k in final_table_ids}

print(len(filtered_implicit_question_ids), "->", len(final_implicit_qids))
print(len(filtered_explicit_answer_mention_ids), "->", len(final_explicit_ansmention_qids))
print(len(filtered_explicit_question_ids), "->", len(final_explicit_qids))
print(len(filtered_tables_dict), "->", len(set(final_tables)))

implicit_questions_array = create_final_questions(final_implicit_qids)
explicit_questions_array = create_final_questions(final_explicit_qids)
relevant_qids = final_implicit_qids + final_explicit_qids + final_explicit_ansmention_qids
print(len(relevant_qids), len(set(relevant_qids)))
final_questions_array = create_final_questions(relevant_qids)
explicit_ans_mention_array = create_final_questions(final_explicit_ansmention_qids)

print(len(implicit_questions_array))
print(len(explicit_questions_array))
print(len(explicit_ans_mention_array))
print(len(final_questions_array))

# WTQ:
    # 2882 -> 2879 IMPLICIT
    # 2777 -> 2773 ANSWER-MENTION QUES
    # 3539 -> 3523 EXPLICIT
    # 1259 -> 1259 TABLES
    # 9175 9175
    # 2879
    # 3523
    # 2773
    # 9175

# FetaQA:
    # 578 -> 612   IMPLICIT
    # 3181 -> 3180 ANSWER-MENTION QUES
    # 2543 -> 2499 EXPLICIT
    # 5908 -> 5898 TABLES
    # 6291 6291
    # 612
    # 2499
    # 3180
    # 6291

# WikiSQL
    # 315 -> 315     IMPLICIT
    # 6374 -> 6374 ANSWER-MENTION QUES
    # 12960 -> 12956 EXPLICIT
    # 9784 -> 9784 TABLES
    # 19645 19645 
    # 315
    # 12956
    # 6374
    # 19645

# HybridQA
    # 17640 -> 17647 IMPLICIT
    # 5130 -> 5130 ANSWER-MENTION QUES
    # 5848 -> 5819 EXPLICIT
    # 8085 -> 8085 TABLES
    # 28596 28596
    # 17647
    # 5819
    # 5130
    # 28596

17640 -> 17647
5130 -> 5130
5848 -> 5819
8085 -> 8085
28596 28596
17647
5819
5130
28596


In [211]:
with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "tables.jsonl"), "w") as f:
    for table_id, table in final_tables.items():
        f.write(json.dumps(table) + "\n")

with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "implicit_questions.jsonl"), "w") as f:
    for question in implicit_questions_array:
        f.write(json.dumps(question) + "\n")
    
with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "explicit_questions.jsonl"), "w") as f:
    for question in explicit_questions_array:
        f.write(json.dumps(question) + "\n")

with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "questions.jsonl"), "w") as f:
    for question in final_questions_array:
        f.write(json.dumps(question) + "\n")

with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "explicit_ans_mention.jsonl"), "w") as f:
    for question in explicit_ans_mention_array:
        f.write(json.dumps(question) + "\n")

In [212]:
relevant_image_ids = []

for table_id, filtered_table in final_tables.items():
    img_ids = get_relevant_images(filtered_table['table_array'])
    relevant_image_ids.extend(img_ids)

In [213]:
file_names_to_variables = [
    "image_id_to_image_path",
    "image_id_to_original_string",
    "image_id_to_qid",
    "image_id_to_wikipedia_link",
]

# Initialize an empty dictionary to hold the loaded data from each file
loaded_data = {}
filtered_data = {}
relevant_image_ids = set(relevant_image_ids)
# Loop through each file, load its contents, and assign it to the corresponding variable
for file_name in file_names_to_variables:
    file_path = os.path.join(BASE_DIR, "final_dataset_filtered", f"{file_name}.json")
    with open(file_path, "r") as f:
        loaded_data[file_name] = json.load(f)
    filtered_data[file_name] = {}
    for img_id in loaded_data[file_name]:
        if img_id in relevant_image_ids:
            filtered_data[file_name][img_id] = loaded_data[file_name][img_id]
    new_file_path = os.path.join(BASE_DIR, "experiment_ready_dataset", f"{file_name}.json")
    with open(new_file_path, "w") as f:
        json.dump(filtered_data[file_name], f, indent=4)

# Now, loaded_data contains all the data from the files, accessible by their variable names
# For example, to access the image path data:
print("Before filtering number of images.", len(loaded_data['image_id_to_image_path']))

print("After filtering number of images.", len(filtered_data['image_id_to_image_path']))

Before filtering number of images. 189320
After filtering number of images. 152300


In [214]:
print(len(filtered_explicit_question_ids))

5848


In [215]:
filtered_explicit_questions_data = {}
for qid in final_explicit_qids:
    qid_data = explicit_questions_data[(qid)]
    if BASE_DIR.split("/")[-1] != "fetaqa_MM_cleaned":
        table_id = questions_dict[(qid)]['table_context']
    else:
        table_id = questions_dict[int(qid)]['table_context']
    table_id = table_id_to_finaltable[table_id]
    # maybe_avoid_cells, definitely_avoid_cells = get_table_explicit_cells(table_id)
    maybe_avoid_cells = tableid_to_maybeavoidcells[table_id]
    
    if BASE_DIR.split("/")[-1] == "WikiSQL":
        definitely_avoid_cells = questions_dict[qid]['evidence']
    else:
        definitely_avoid_cells = tableid_to_definitelyavoidcells[table_id]
    table = filtered_tables_dict[table_id]['table_array']

    is_found = False
    for cell in definitely_avoid_cells:
        if is_cell_image(table[cell[0]][cell[1]]):
            is_found = True
    
    # for cell in maybe_avoid_cells:
    #     if is_cell_image(table[cell[0]][cell[1]]):
    #         is_found = True
            # print("AAAAAAAAAAAAAAAAAA")
            # print(qid)
            # print(cell)
            # print(table[cell[0]][cell[1]])
            # print(table_id)
            # print(table)
    
    # for dat in qid_data:
    #     if dat['img_id'] in relevant_image_ids:
    #         is_found = True
    #         break
    # for dat in qid_data:
    #     cell_idx = dat['cell_idx']
    #     if is_cell_image(table[cell_idx[0]][cell_idx[1]]):
    #         is_found = True
    if not is_found:
        print("AAAAAAAAAAAAAAAAAA")

print(len(filtered_explicit_question_ids))

AAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAA
5848


In [216]:
filtered_explicit_questions_data = {}
for k in final_explicit_qids:
    explicit_ques_data = explicit_questions_data[str(k)]
    if BASE_DIR.split("/")[-1] != "fetaqa_MM_cleaned":
        table_id = questions_dict[(k)]['table_context']
    else:
        table_id = questions_dict[int(k)]['table_context']
    table_id = table_id_to_finaltable[table_id]
    new_explicit_ques_data = []
    for _  in explicit_ques_data:
        cell = _['cell_idx']
        if is_cell_image(filtered_tables_dict[table_id]['table_array'][cell[0]][cell[1]]):
            new_explicit_ques_data.append(_['cell_idx'])
        else:
            print("AAAAAAA")
    
    filtered_explicit_questions_data[k] = {"table_id": table_id, "cell_indices": new_explicit_ques_data}

AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA


In [217]:
with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "explicit_questions_data.json"), "w") as f:
    json.dump(filtered_explicit_questions_data, f, indent=4)

In [218]:
filtered_ansmention_data = {}
for k in final_explicit_ansmention_qids:
    explicit_ques_data = explicit_answer_mention_data[str(k)]
    if BASE_DIR.split("/")[-1] != "fetaqa_MM_cleaned":
        table_id = questions_dict[str(k)]['table_context']
    else:
        table_id = questions_dict[int(k)]['table_context']
    table_id = table_id_to_finaltable[table_id]
    new_explicit_ques_data = []
    for _  in explicit_ques_data:
        cell = _['cell_idx']
        if is_cell_image(filtered_tables_dict[table_id]['table_array'][cell[0]][cell[1]]):
            new_explicit_ques_data.append(_['cell_idx'])
    
    filtered_ansmention_data[k] = {"table_id": table_id, "cell_indices": new_explicit_ques_data}

In [219]:
with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "explicit_answer_mention_data.json"), "w") as f:
    json.dump(filtered_ansmention_data, f, indent=4)

In [220]:
##########Sanity check
import os
import json

with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "explicit_questions_data.json")) as f:
    filtered_explicit_questions_data = json.load(f)

tables_dict = {}

with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "tables.jsonl")) as f:
    for line in f.readlines():
        json_line = json.loads(line)
        tables_dict[json_line['table_id']] = json_line

In [221]:
for qid, explicit_ques_data in filtered_explicit_questions_data.items():
    table_id = explicit_ques_data['table_id']
    cell_indices = explicit_ques_data['cell_indices']
    table_array = tables_dict[table_id]['table_array']
    if len(cell_indices)==0:
        print("OH NOOOOOOOOOO!")
    for cell in cell_indices:
        if not is_cell_image(table_array[cell[0]][cell[1]]):
            print("OMG!!")

In [222]:

def is_valid(table_array):
    if len(table_array) > 55:
        return False

    col_count = [0]*len(table_array[0])

    for row in table_array:
        for col_idx, cell in enumerate(row):
            if len(get_all_image_tags(cell)) > 0:
                col_count[col_idx] += 1
    
    col_count = [x/len(table_array) for x in col_count]

    if max(col_count) < 0.3:
        print(max(col_count))
        print(len(table_array)-1)
        return False

    if max(col_count) > 0.75:
        print(max(col_count))
        return False

    return True

In [223]:
for table_id, table_info in tables_dict.items():
    table_array = table_info['table_array']
    
    if not is_valid(table_array):
        print("OMG!!")

In [224]:
questions_dict = {}
with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "implicit_questions.jsonl")) as f:
    for line in f.readlines():
        json_line = json.loads(line)
        questions_dict[json_line['question_id']] = json_line

print(list(questions_dict.items())[:10])

[('cc80bec0733f4c5b', {'question_id': 'cc80bec0733f4c5b', 'question': "What year was the worst rated ( according to Rotten Tomatoes ) Child 's Play franchise film released ?", 'answer-text': '1998', 'question_postag': 'WP NN VBD DT JJS VBN ( VBG TO NNP NNP ) NNP POS NNP NN NN VBN .', 'answer-node': [['Bride of Chucky', [3, 0], '/wiki/Bride_of_Chucky', 'passage'], ['Seed of Chucky', [4, 0], '/wiki/Seed_of_Chucky', 'passage']], 'table_context': "Child's_Play_(film_series)_0"}), ('10a842047fcaa039', {'question_id': '10a842047fcaa039', 'question': "What year was the Child 's Play franchise film released that recieved 33 % on Rotten Tomatoes ?", 'answer-text': '2004', 'question_postag': 'WP NN VBD DT NNP POS NNP NN NN VBD IN VBD CD NN IN NNP NNP .', 'answer-node': [['Bride of Chucky', [3, 0], '/wiki/Bride_of_Chucky', 'passage'], ['Seed of Chucky', [4, 0], '/wiki/Seed_of_Chucky', 'passage']], 'table_context': "Child's_Play_(film_series)_0"}), ('60d2b64d9978f589', {'question_id': '60d2b64d997

In [225]:
if BASE_DIR.split("/")[-1] == "WikiTableQuestions":
    with open(os.path.join(BASE_DIR, "final_dataset_filtered", "qid_to_evidence_cells.json")) as f:
        implicit_questions_evidence_cells = json.load(f)

In [226]:
problematic_qid = set()
for qid, question_data in questions_dict.items():
    table_id = question_data['table_context']
    table_array = tables_dict[table_id]['table_array']
    
    if BASE_DIR.split("/")[-1] == "WikiTableQuestions":
        evidence_cells = implicit_questions_evidence_cells[qid]
    elif BASE_DIR.split("/")[-1] == "WikiSQL":
        evidence_cells = question_data['weak_evidence_idx']
    elif BASE_DIR.split("/")[-1] == "Hybrid_QA_MM":
        evidence_cells = question_data["answer-node"]
        evidence_cells = [tuple([cell[1][0]+1, cell[1][1]]) for cell in evidence_cells]
    else:
        evidence_cells = question_data['highlighted_cell_ids']
    
    is_found = False
    for cell in evidence_cells:
        try:
            if is_cell_image(table_array[cell[0]][cell[1]]):
                is_found = True
                break
        except Exception as e:
            continue
    if not is_found:
        problematic_qid.add(qid)
        print("---------------")
        for cell in evidence_cells:
            print(cell)
            try:
                print(table_array[cell[0]][cell[1]])
            except:
                print("AAAAAAAAAAAA")
        

In [227]:
##########Sanity check
import os
import json

with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "explicit_answer_mention_data.json")) as f:
    filtered_explicit_questions_data = json.load(f)

tables_dict = {}

with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "tables.jsonl")) as f:
    for line in f.readlines():
        json_line = json.loads(line)
        tables_dict[json_line['table_id']] = json_line

In [228]:
for qid, explicit_ques_data in filtered_explicit_questions_data.items():
    table_id = explicit_ques_data['table_id']
    cell_indices = explicit_ques_data['cell_indices']
    table_array = tables_dict[table_id]['table_array']
    if len(cell_indices)==0:
        print("OH NOOOOOOOOOO!")
    for cell in cell_indices:
        if not is_cell_image(table_array[cell[0]][cell[1]]):
            print("OMG!!")

### Visualization

In [229]:
questions_dict = {}
with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "questions.jsonl")) as f:
    for line in f.readlines():
        line = json.loads(line)
        questions_dict[line['question_id']] = line

with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "explicit_answer_mention_data.json")) as f:
    ans_mention_ques = json.load(f)

with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "explicit_questions_data.json")) as f:
    explicit_ques = json.load(f)

with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "tables.jsonl")) as f:
    tables_dict = {}
    for line in f.readlines():
        table = json.loads(line)
        tables_dict[table['table_id']] = table

implicit_qids = []
with open(os.path.join(BASE_DIR, "experiment_ready_dataset", "implicit_questions.jsonl")) as f:
    for line in f.readlines():
        line = json.loads(line)
        implicit_qids.append(line['question_id'])

if BASE_DIR.split("/")[-1] == "WikiTableQuestions":
    with open(os.path.join(BASE_DIR, "final_dataset_filtered", "qid_to_evidence_cells.json")) as f:
        qid_to_highlighted_cells = json.load(f)

In [230]:
def generate_table_string(table_array):
    table_string = ""
    
    for row_idx, row in enumerate(table_array):
        if row_idx == 0:
            # table_string = table_string + "0"
            # for i in range(len(row)):
            #     table_string = table_string + str(i+1) + " | "
            # table_string = table_string + "\n"
            pass
        
        for col_idx, cell in enumerate(row):
            if col_idx == 0:
                pass
                # if row_idx == 0:
                #     pass
                #     table_string = table_string + "index"
                # else:
                #     table_string = table_string + str(row_idx-1)

            cell = cell.replace("\t", " ").replace("\n", " ").replace("\\n", " ").replace("\\t", " ").replace("|", " ")
            table_string = table_string + cell + f" {row_idx, col_idx}" +  " | " 
        table_string = table_string + "\n"
    
    return table_string


In [231]:
list(questions_dict.items())[:10]

[('cc80bec0733f4c5b',
  {'question_id': 'cc80bec0733f4c5b',
   'question': "What year was the worst rated ( according to Rotten Tomatoes ) Child 's Play franchise film released ?",
   'answer-text': '1998',
   'question_postag': 'WP NN VBD DT JJS VBN ( VBG TO NNP NNP ) NNP POS NNP NN NN VBN .',
   'answer-node': [['Bride of Chucky',
     [3, 0],
     '/wiki/Bride_of_Chucky',
     'passage'],
    ['Seed of Chucky', [4, 0], '/wiki/Seed_of_Chucky', 'passage']],
   'table_context': "Child's_Play_(film_series)_0"}),
 ('10a842047fcaa039',
  {'question_id': '10a842047fcaa039',
   'question': "What year was the Child 's Play franchise film released that recieved 33 % on Rotten Tomatoes ?",
   'answer-text': '2004',
   'question_postag': 'WP NN VBD DT NNP POS NNP NN NN VBD IN VBD CD NN IN NNP NNP .',
   'answer-node': [['Bride of Chucky',
     [3, 0],
     '/wiki/Bride_of_Chucky',
     'passage'],
    ['Seed of Chucky', [4, 0], '/wiki/Seed_of_Chucky', 'passage']],
   'table_context': "Child's_P

In [233]:
ex_list = list(explicit_ques.items())
random.shuffle(ex_list)
for qid, q_data in ex_list[:20]:
    print("--------------------------------")
    print(qid)
    table_array = tables_dict[q_data['table_id']]['table_array']
    og_table_array = tables_dict[q_data['table_id']]['original_table_array']
    table_strings = [og_table_array[_[0]][_[1]] for _ in q_data['cell_indices']]
    print(table_strings)
    print(q_data['cell_indices'])
    table_string = generate_table_string(table_array)
    print(table_string)
    ques = questions_dict[(qid)]
    print(ques['question'])
    print(ques['answer-text'])
    print("--------------------------------")
    

--------------------------------
69a8c7bd36449801
['Ljubljana , Slovenia', 'Ljubljana , Slovenia', 'Ljubljana , Slovenia', 'Ljubljana , Slovenia']
[[6, 3], [6, 3], [7, 3], [7, 3]]
Name (0, 0) | Group Level (0, 1) | Improv Type (0, 2) | Location (0, 3) | Date Established (0, 4) | 
Yours Truly Theatre (1, 0) | Professional (1, 1) | Multiple (1, 2) | {IMG-{HTQ-2130-1-3-0}} , {IMG-{HTQ-2130-1-3-1}} (1, 3) | 2003 (1, 4) | 
Improv Bandits (2, 0) | Professional (2, 1) | Multiple (2, 2) | {IMG-{HTQ-2130-2-3-0}} , {IMG-{HTQ-2130-2-3-1}} (2, 3) | 1997 (2, 4) | 
The Court Jesters (3, 0) | Professional (3, 1) | Multiple (3, 2) | {IMG-{HTQ-2130-3-3-0}} , {IMG-{HTQ-2130-3-3-1}} (3, 3) | 1989 (3, 4) | 
Wellington Improvisation Troupe (4, 0) | Professional (4, 1) | Multiple (4, 2) | {IMG-{HTQ-2130-4-3-0}} , {IMG-{HTQ-2130-4-3-1}} (4, 3) | 2004 (4, 4) | 
boom chicago (5, 0) | Professional (5, 1) | Multiple (5, 2) | amsterdam , netherlands (5, 3) | 1993 (5, 4) | 
IGLU Theatre (6, 0) | Professional (6, 1

In [194]:
ex_list = list(ans_mention_ques.items())
random.shuffle(ex_list)
for qid, q_data in ex_list[:20]:
    print("-------------------------")
    print(qid)
    table_array = tables_dict[q_data['table_id']]['table_array']
    og_table_array = tables_dict[q_data['table_id']]['original_table_array']
    table_strings = [og_table_array[_[0]][_[1]] for _ in q_data['cell_indices']]
    print(table_strings)
    print(q_data['cell_indices'])
    table_string = generate_table_string(table_array)
    print(table_string)
    ques = questions_dict[(qid)]
    print(ques['question'])
    print(ques['answer'])
    print("-------------------------")

-------------------------
WSQL-80058
['{IMG-{WSQ-2-13312898-17-6-5-0}}']
[[6, 5]]
Round (0, 0) | Pick # (0, 1) | Overall (0, 2) | Name (0, 3) | Position (0, 4) | College (0, 5) | 
1 (1, 0) | 25 (1, 1) | 25 (1, 2) | Eddie Hinton (1, 3) | Wide Receiver (1, 4) | {IMG-{WSQ-2-13312898-17-1-5-0}} (1, 5) | 
2 (2, 0) | 7 (2, 1) | 33 (2, 2) | {IMG-{WSQ-2-13312898-17-2-3-0}} (2, 3) | Linebacker (2, 4) | {IMG-{WSQ-2-13312898-17-2-5-0}} (2, 5) | 
2 (3, 0) | 25 (3, 1) | 51 (3, 2) | Tom Maxwell (3, 3) | Defensive Back (3, 4) | {IMG-{WSQ-2-13312898-17-3-5-0}} (3, 5) | 
3 (4, 0) | 25 (4, 1) | 77 (4, 2) | Dennis Nelson (4, 3) | Offensive Tackle (4, 4) | Illinois State (4, 5) | 
4 (5, 0) | 9 (5, 1) | 87 (5, 2) | Jackie Stewart (5, 3) | Running Back (5, 4) | {IMG-{WSQ-2-13312898-17-5-5-0}} (5, 5) | 
5 (6, 0) | 25 (6, 1) | 129 (6, 2) | King Dunlap (6, 3) | Defensive Tackle (6, 4) | {IMG-{WSQ-2-13312898-17-6-5-0}} (6, 5) | 
6 (7, 0) | 24 (7, 1) | 154 (7, 2) | Bill Fortier (7, 3) | Guard (7, 4) | LSU (7, 5)

In [198]:
random.shuffle(implicit_qids)
for qid in implicit_qids[:20]:
    print("---------------")
    q_data = questions_dict[qid]
    print(qid)
    table_array = tables_dict[q_data['table_context']]['table_array']
    og_table_array = tables_dict[q_data['table_context']]['original_table_array']
    table_strings = []
    if BASE_DIR.split("/")[-1] == "WikiTableQuestions":
        for row,col in qid_to_highlighted_cells[qid]:
            if 0 <= row < len(og_table_array) and 0 <= col < len(og_table_array[0]):
                table_strings.append(og_table_array[row][col])
        print(table_strings)
        print(qid_to_highlighted_cells[qid])
    elif BASE_DIR.split("/")[-1] == "WikiSQL":
        ev_cells = q_data['weak_evidence_idx']
        table_strings = [og_table_array[cell[0]][cell[1]] for cell in ev_cells if len(get_all_image_tags(table_array[cell[0]][cell[1]])) > 0]
        ev_cells = [cell for cell in ev_cells if len(get_all_image_tags(table_array[cell[0]][cell[1]])) > 0]
        print(table_strings)
        print(ev_cells)
    else:
        ev_cells = q_data['highlighted_cell_ids']
        table_strings = [og_table_array[cell[0]][cell[1]] for cell in ev_cells if len(get_all_image_tags(table_array[cell[0]][cell[1]])) > 0]
        ev_cells = [cell for cell in ev_cells if len(get_all_image_tags(table_array[cell[0]][cell[1]])) > 0]
        print(table_strings)
        print(ev_cells)
    table_string = generate_table_string(table_array)
    print(table_string)
    ques = questions_dict[(qid)]
    print(ques['question'])
    print(ques['answer'])
    print("---------------")

---------------
WSQL-51579
['{IMG-{WSQ-2-1226454-4-1-0-0}}']
[[1, 0]]
Year (0, 0) | Team (0, 1) | Chassis (0, 2) | Engine (0, 3) | Rank (0, 4) | Points (0, 5) | 
{IMG-{WSQ-2-1226454-4-1-0-0}} (1, 0) | Forsythe Newman (1, 1) | March 82C (1, 2) | Ford Cosworth DFX (1, 3) | 22nd (1, 4) | 28 (1, 5) | 
{IMG-{WSQ-2-1226454-4-2-0-0}} (2, 0) | Shierson Racing (2, 1) | DSR-1 (2, 2) | Ford Cosworth DFX (2, 3) | 4th (2, 4) | 110 (2, 5) | 
{IMG-{WSQ-2-1226454-4-3-0-0}} (3, 0) | Shierson Racing (3, 1) | Lola T800 (3, 2) | Ford Cosworth DFX (3, 3) | 4th (3, 4) | 110 (3, 5) | 
{IMG-{WSQ-2-1226454-4-4-0-0}} (4, 0) | Team Penske (4, 1) | March 85C (4, 2) | Ford Cosworth DFX (4, 3) | 4th (4, 4) | 126 (4, 5) | 
1986 (5, 0) | Team Penske (5, 1) | March 86C (5, 2) | Ford Cosworth DFX (5, 3) | 3rd (5, 4) | 147 (5, 5) | 
1986 (6, 0) | Team Penske (6, 1) | Penske PC-15 (6, 2) | Chevrolet 265A (6, 3) | 3rd (6, 4) | 147 (6, 5) | 
{IMG-{WSQ-2-1226454-4-7-0-0}} (7, 0) | Team Penske (7, 1) | Penske PC-16 (7, 2) | 